In [4]:
"""
定义层和块
"""
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch. rand(2, 20)
net(X)


tensor([[ 0.1749,  0.1689, -0.0624, -0.0513,  0.0611,  0.3318,  0.2474,  0.0919,
         -0.1124,  0.0984],
        [ 0.1989,  0.1400,  0.0878,  0.0598,  0.1960,  0.2654,  0.1280, -0.0049,
         -0.1765,  0.0356]], grad_fn=<AddmmBackward0>)

In [8]:
"""
自定义块
"""
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
        
    """
    前向计算
    """
    def forward(self, x):
        return self.out(F.relu(self.hidden(X)))
net = MLP()
net(X) # 调用了net.__call__(X)

tensor([[ 0.0186, -0.0548, -0.0991,  0.1206,  0.0581, -0.3094, -0.0835, -0.2412,
         -0.0114,  0.0744],
        [ 0.1267,  0.0635, -0.1220,  0.1154, -0.0066, -0.1424, -0.0652, -0.1659,
         -0.0141,  0.2454]], grad_fn=<AddmmBackward0>)

In [14]:
"""
定义顺序块
 def __init__(self， *args) 中有无*args的区别
 若有，实例化的时候属性值不能为空， 反之则可以为空
"""
class MySequential(nn.Module):
    def __init__(self, *args): # 此时按照顺序传入这个类中
        super().__init__() # 继承父类
        for block in args:
            self._modules[block] = block # 此时为按序字典 bhlock 本身作为它的key，存在_modules里面的为层，以字典的形式
            
    def forward(self, X):
        for block in self._modules.values():
            X  = block(X) # 按照顺序返回
        return X 
            
net = MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
X = torch.rand(2,20)
net(X)   

tensor([[ 0.1444, -0.0263, -0.1959, -0.1037, -0.1316,  0.0897,  0.3016, -0.0779,
          0.1613,  0.1671],
        [ 0.0041, -0.0932, -0.0388, -0.0372, -0.1574,  0.1542,  0.1882, -0.0645,
          0.1474,  0.1953]], grad_fn=<AddmmBackward0>)

In [20]:
"""
正向传播
"""
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        """
        backward 可以追踪这个参数并且计算它的梯度 在写代码的过程中
        不要把网络的输入和 Ground Truth 的 requires_grad 设置为 True
        """
        self.rand_weight = torch.rand((20, 20), requires_grad = False) 
        self.linear = nn.Linear(20, 20)
        
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
    
net = FixedHiddenMLP()
net(X)
        

tensor(-0.0961, grad_fn=<SumBackward0>)

In [27]:
"""
混合各种块在一起使用
"""
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20,64),nn.ReLU(),
                                nn.Linear(64,32),nn.ReLU())
        self.linear = nn.Linear(32,16)
        
    def forward(self, X):
        return self.linear(self.net(X))
    
chimear = nn.Sequential(NestMLP(),nn.Linear(16,20),FixedHiddenMLP())
X = torch.rand(2,20)
chimear(X)

tensor(-0.0566, grad_fn=<SumBackward0>)

In [28]:
# 首先关注具有单隐藏层的多层感知机
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1)) # net是一个python的list
X = torch.rand(size=(2,4))
print(net(X))

print(net[2].state_dict()) # 访问参数，net[2]就是最后一个输出层
print(type(net[2].bias)) # 目标参数 
print(net[2].bias)
print(net[2].bias.data)
print(net[2].weight.grad == None) # 还没进行反向计算，所以grad为None
print(*[(name, param.shape) for name, param in net[0].named_parameters()])  # 一次性访问所有参数  *解元组      
print(*[(name, param.shape) for name, param in net.named_parameters()])  # 0是第一层名字，1是ReLU，它没有参数
print(net.state_dict()['2.bias'].data) # 通过名字获取参数

tensor([[-0.0983],
        [-0.2655]], grad_fn=<AddmmBackward0>)
OrderedDict([('weight', tensor([[-0.2916,  0.2529, -0.0201, -0.2267,  0.0269, -0.0150, -0.1336, -0.1406]])), ('bias', tensor([-0.3045]))])
<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.3045], requires_grad=True)
tensor([-0.3045])
True
('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))
tensor([-0.3045])


In [29]:
"""
嵌套快
"""
# 从嵌套块收集参数
def block1():
    return nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,4),nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block{i}',block1()) # f'block{i}' 可以传一个字符串名字过来，block2可以嵌套四个block1                                      
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4,1))
print(rgnet(X))
print(rgnet)

tensor([[-0.2849],
        [-0.2849]], grad_fn=<AddmmBackward0>)
Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [30]:
"""内置初始化"""
net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))

def init_normal(m): # m是一个Moudel
    if type(m) == nn.Linear: # 如果m是线性层
        nn.init.normal_(m.weight, mean=0, std=0.01) # 下划线表示把m.weight的值替换掉   添加下划线为原地操作 直接修改torch的内容 节省内存
        nn.init.zeros_(m.bias)
        
net.apply(init_normal) # 会递归调用 直到所有层都初始化 类似于深度优先算法
print(net[0].weight.data[0])
print(net[0].bias.data[0])

tensor([-0.0008,  0.0039,  0.0094,  0.0038])
tensor(0.)


In [31]:
# 对某些块应用不同的初始化
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)    
        
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)
        
net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.1266,  0.6019,  0.3107, -0.3287])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [32]:
"""
参数替换
"""
# 自定义初始化
def my_init(m):
    if type(m) == nn.Linear:
        print("Init",*[(name, param.shape) for name, param in m.named_parameters()][0])  # 打印名字是啥，形状是啥       
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >=  5 # 这里*=的代码相当于先计算一个布尔矩阵(先判断>=)，然后再用布尔矩阵的对应元素去乘以原始矩阵的每个元素。保留绝对值大于5的权重，不是的话就设为0

net.apply(my_init)
print(net[0].weight[:2])
net[0].weight.data[:] += 1 # 所有的数值加一 参数替换
net[0].weight.data[0,0] = 42
print(net[0].weight.data[0])

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])
tensor([[ 0.0000, -9.4000,  8.0926, -0.0000],
        [ 7.5735, -0.0000, -5.7080,  6.8718]], grad_fn=<SliceBackward0>)
tensor([42.0000, -8.4000,  9.0926,  1.0000])


In [33]:
# 参数绑定 数据共享
shared = nn.Linear(8,8) # shared层
net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),shared,nn.ReLU(),
                    shared,nn.ReLU(),nn.Linear(8,1))  # 第2个隐藏层和第3个隐藏层是share权重的，第一个和第四个是自己的  
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0]) # net[2]是第一个shared, 
net[2].weight.data[0,0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


In [39]:
"""
自定义层
构造一个没有任何参数的自定义层
"""
import torch
import torch.nn.functional as F
from torch import nn


class C(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, X):
        return X - X.mean()
    
layer = C()
layer = (torch.FloatTensor([1, 2, 3, 4, 5]))

# 将层作为组件合并到构建更复杂的模型中
net = nn.Sequential(nn.Linear(8,128),C())
Y = net(torch.rand(4,8))
print(Y.mean())



# 带参数的图层
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units,units)) # nn.Parameter使得这些参数加上了梯度    
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)
    
dense = MyLinear(5,3)
print(dense.weight)

# 使用自定义层直接执行正向传播计算
print(dense(torch.rand(2,5)))
# 使用自定义层构建模型
net = nn.Sequential(MyLinear(64,8),MyLinear(8,1))
print(net(torch.rand(2,64)))

tensor(2.3283e-09, grad_fn=<MeanBackward0>)
Parameter containing:
tensor([[-1.3157,  0.7079,  1.2524],
        [ 0.3657, -0.1251,  0.8822],
        [-0.9113, -0.6550, -0.0930],
        [ 1.0832,  1.2037,  0.6270],
        [-0.2388,  0.8205,  0.7066]], requires_grad=True)


In [45]:
"""
读写文件
"""
# 加载和保存张量
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')
x2 = torch.load("x-file")
print(x2)

#存储一个张量列表，然后把它们读回内存
y = torch.zeros(4)
torch.save([x,y],'x-files')
x2, y2 = torch.load('x-files')
print(x2)
print(y2)

# 写入或读取从字符串映射到张量的字典
mydict = {'x':x,'y':y}
torch.save(mydict,'mydict')
mydict2 = torch.load('mydict')
print(mydict2)

# 加载和保存模型参数
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20,256)
        self.output = nn.Linear(256,10)
    
    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))
    
net = MLP()
X = torch.randn(size=(2,20))
Y = net(X)

# 将模型的参数存储为一个叫做"mlp.params"的文件
torch.save(net.state_dict(),'mlp.params')

# 实例化了原始多层感知机模型的一个备份。直接读取文件中存储的参数
clone = MLP() # 必须要先声明一下，才能导入参数 此时clone的参数是随机初始化的
clone.load_state_dict(torch.load("mlp.params")) # 将保存的参数协会
print(clone.eval()) # eval()是进入测试模式

Y_clone = clone(X)
print(Y_clone == Y)

tensor([0, 1, 2, 3])
tensor([0, 1, 2, 3])
tensor([0., 0., 0., 0.])
{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}
MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)
  ####
tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])
